# Reaction fluxes and compound usage

by Daniel Cuevas

## Introduction

In this notebook we will highlight features in _PyFBA_ that allows the user to obtain a model's reaction distribution after performing flux-balance analysis.

---

**Data used in this notebook**
- _Citrobacter sedlakii_ genome-scale metabolic model
- LB media
---

In [1]:
import sys
import os

# Add local PyFBA to sys path so we can use the developer version
sys.path.insert(0, os.path.expanduser("~") + "/database/ModelSEEDDatabase/")

import PyFBA

## Load metabolic model

_PyFBA_ contains two methods to load and save metabolic models.
1. Metabolic model directories
2. SBML files

## Specify location of metabolic model
You can specify the location of an existing model on your system, or generate a gap-filled model  using the Gap-fill_a_model notebook and an assigned_functions file.

In [2]:
modelDir = "saved_models/"
orgName = "Citrobacter_sedlakii"

model = PyFBA.model.load_model(modelDir, orgName)
print("Number of reactions", len(model.reactions))
print("Number of compounds", len(model.compounds))

Number of reactions 1528
Number of compounds 1428


In [4]:
# Test its ability to grow in the LB media "ArgonneLB.txt"
status, biomassFlux, growth = model.run_fba(media_file="ArgonneLB.txt")
print("Biomass flux: {:.3f}".format(biomassFlux))
print("Growth:", growth)

Biomass flux: 358.249
Growth: True


## Obtain flux distribution from reactions

_PyFBA_ provides the capability to obtain the flux distribution resulting from a flux-balance analysis process. Using the `PyFBA.model.model_reaction_fluxes()` function, each reaction and their flux is returned as a Dictionary where:
- **KEY:** ModelSEED Reaction ID (String)
- **VALUE:** Reaction flux (float)

In [6]:
#biomass_equation = PyFBA.metabolism.biomass_equation("gramnegative")
rxnFluxes = PyFBA.model.model_reaction_fluxes(model=model, media_file="ArgonneLB.txt")

In [7]:
# Get number of reactions with a non-zero flux
nonzeroRxns = {r: flux for r, flux in rxnFluxes.items() if flux != 0}
print("Number of non-zero flux reactions:", len(nonzeroRxns), "out of", len(rxnFluxes))
print("Percent of reactions with non-zero flux: {:.2f}%".format(len(nonzeroRxns) / len(rxnFluxes) * 100))

Number of non-zero flux reactions: 422 out of 1701
Percent of reactions with non-zero flux: 24.81%


### Uptake and secretion reactions

The uptake and secretion reactions are designated as "reaction sinks". In practice they are indicated as **EX** reactions or with the SBML attribute **boundaryCondition** set to _True_. These reaction sinks are necessary to satisfy the steady-state assumption of flux-balance analysis.

In _PyFBA_, we add these reactions when flux-balance analysis is executed. These are added based on all external compounds within the system:
1. Media compounds
2. External compounds from transports

The flux values for these reaction sinks indicate the compounds transported into the cell from the media (**negative flux for uptake**) and the compounds secreted to the environment (**positive values for secretion**).

In [8]:
# Get uptake/secretion reactions
usRxns = {r: flux for r, flux in rxnFluxes.items() if "UPTAKE" in r}
print("Number of uptake/secretion reactions:", len(usRxns))

Number of uptake/secretion reactions: 172


In [9]:
# Print out uptake/secretion fluxes
# Negative values indicate usage by the cell (UPTAKE)
# Positive values indicate secretion by the cell (SECRETION)
for r, flux in usRxns.items():
    threshold = 0.000001
    # Skip very small numbers
    if flux > -threshold and flux < threshold:
        continue
        
    print("{}:\t{:.3f}".format(r, flux))

UPTAKE_SECRETION_REACTION cpd00039:	-102.449
UPTAKE_SECRETION_REACTION cpd00069:	-41.235
UPTAKE_SECRETION_REACTION cpd00027:	-955.495
UPTAKE_SECRETION_REACTION cpd00156:	654.524
UPTAKE_SECRETION_REACTION cpd00041:	1000.000
UPTAKE_SECRETION_REACTION cpd00205:	-2.788
UPTAKE_SECRETION_REACTION cpd00058:	-2.788
UPTAKE_SECRETION_REACTION cpd00035:	1000.000
UPTAKE_SECRETION_REACTION cpd00161:	-950.677
UPTAKE_SECRETION_REACTION cpd00107:	-915.679
UPTAKE_SECRETION_REACTION cpd00048:	-35.693
UPTAKE_SECRETION_REACTION cpd00226:	-112.440
UPTAKE_SECRETION_REACTION cpd00100:	1000.000
UPTAKE_SECRETION_REACTION cpd00067:	908.039
UPTAKE_SECRETION_REACTION cpd00034:	-2.788
UPTAKE_SECRETION_REACTION cpd00218:	-5.575
UPTAKE_SECRETION_REACTION cpd00092:	-73.192
UPTAKE_SECRETION_REACTION cpd00030:	-2.788
UPTAKE_SECRETION_REACTION cpd00129:	-66.168
UPTAKE_SECRETION_REACTION cpd00254:	-2.788
UPTAKE_SECRETION_REACTION cpd11416:	358.249
UPTAKE_SECRETION_REACTION cpd00065:	-16.942
UPTAKE_SECRETION_REACTION cpd0